In [1]:
# Import libraries
import torch
import torch.nn as nn
import numpy as np
from pydmd import DMD

In [2]:
# Import data from data/PSEI.csv to numpy array
# Get only the closing price (usecols=4)
data = np.genfromtxt('data/PSEI.csv', delimiter=',', skip_header=1, usecols=4)
print(f"Length of data: {len(data)}")
print(f"Closing Prices: {data}")

Length of data: 5675
Closing Prices: [2141.77 2153.18 2074.75 ... 6923.08 6842.79 6876.79]


In [3]:
# Split data into training and testing
len_train = int(len(data) * 0.80)
data_train = data[:len_train]
print(f"Length of data_train: {len(data_train)}\nData Train: {data_train}\n")
data_test = data[len_train:]
print(f"Length of data_test: {len(data_test)}\nData Test: {data_test}")

Length of data_train: 4540
Data Train: [2141.77 2153.18 2074.75 ... 7267.34 7348.42 7233.57]

Length of data_test: 1135
Data Test: [7186.71 7186.62 7233.29 ... 6923.08 6842.79 6876.79]


In [4]:
# Hyperparameters
input_dim = 3
hidden_dim = 5
num_layers = 1
output_dim = 1
num_epochs = 100
learning_rate = 0.01

In [5]:
X = np.zeros(((len(data_train)-(input_dim-1)), input_dim))
for i in range(len(X)):
    X[i, 0:input_dim] = data_train[i:i+input_dim]
print(f"X: {X[0]}")
print(f"X.shape: {X.shape}")

X: [2141.77 2153.18 2074.75]
X.shape: (4538, 3)


In [6]:
dmd = DMD(svd_rank=2, exact=True)
dmd.fit(X)
dynamic_mode = dmd.modes
print(f"Dynamic Mode: {dynamic_mode}")
print(f"Length of Dynamic Mode: {dynamic_mode.shape}")

Dynamic Mode: [[-0.00667728 -0.02347301]
 [-0.00671106  0.00118711]
 [-0.00676263  0.0044092 ]
 ...
 [-0.02346365  0.01133992]
 [-0.02373525  0.02375659]
 [-0.02331795 -0.03459818]]
Length of Dynamic Mode: (4538, 2)


In [7]:
# prepare data for LSTM input
for i in range(len(X)):
    X[i, input_dim:] = dynamic_mode[i, 0]
y = data_train[input_dim:]
print(f"X: {X}")
print(f"y: {len(y)}")

X: [[2141.77 2153.18 2074.75]
 [2153.18 2074.75 2079.11]
 [2074.75 2079.11 2094.29]
 ...
 [7193.68 7227.96 7267.34]
 [7227.96 7267.34 7348.42]
 [7267.34 7348.42 7233.57]]
y: 4537


In [8]:
# set up LSTM model
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x.view(len(x), 1, -1))
        y_pred = self.fc(lstm_out[-1].view(1, -1))
        return y_pred.view(-1)

In [9]:
# convert data to PyTorch tensors
X = torch.Tensor(X)
y = torch.Tensor(y)

In [10]:
# set up LSTM model and optimizer
model = LSTM(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# train the LSTM model
for epoch in range(num_epochs):
    optimizer.zero_grad()
    y_pred = model(X)
    loss = nn.MSELoss()(y_pred, y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss {loss.item()}")

c:\Users\johnm\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([4537])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Loss 21172332.0
Epoch 10, Loss 21170174.0
Epoch 20, Loss 21168012.0
Epoch 30, Loss 21165854.0
Epoch 40, Loss 21163694.0
Epoch 50, Loss 21161532.0
Epoch 60, Loss 21159374.0
Epoch 70, Loss 21157214.0
Epoch 80, Loss 21155056.0
Epoch 90, Loss 21152900.0


In [12]:
# make a prediction for the next closing price
X_test = np.zeros((1, input_dim))
X_test[0, 0:input_dim] = data_test[0:input_dim]
X_test[0, input_dim:] = dynamic_mode[-1, 0]
X_test = torch.Tensor(X_test)
y_pred = model(X_test)
print(f"Predicted closing price: {y_pred.item()}")
print(f"Actual closing price: {data_test[input_dim]}")

Predicted closing price: 2.222327470779419
Actual closing price: 7333.73


In [18]:
import matplotlib.pyplot as plt

# Predict all X
X_pred = np.zeros((len(data_train), input_dim))
X_pred[0, 0:input_dim] = data_train[0:input_dim]
X_pred[0, input_dim:] = dynamic_mode[0, 0]
for i in range(1, len(X_pred)):
    X_pred[i, 0:input_dim] = X_pred[i-1, 1:input_dim+1]
    X_pred[i, input_dim:] = dynamic_mode[i-1, 0]
X_pred = torch.Tensor(X_pred)
y_pred = model(X_pred)
print(f"Predicted closing price: {y_pred}")

ValueError: could not broadcast input array from shape (2,) into shape (3,)